In [1]:
import imagej
import pandas as pd
from scyjava import jimport, config
from glob import glob
import os
import re
import xlsxwriter
from openpyxl import Workbook
from openpyxl import load_workbook

In [2]:
if 'ij' in locals() and ij is not None:
    ij.dispose()
plugins_dir = r'C:\Users\Leonard Piroska\Fiji\Fiji.app\plugins'
config.add_option(f'-Dplugins.dir={plugins_dir}')
ij = imagej.init(r'C:\Users\Leonard Piroska\Fiji\Fiji.app', mode=imagej.Mode.INTERACTIVE)
print(ij.getVersion())
ij.ui().showUI()

#import imageJ
ImagePlus=jimport('ij.ImagePlus')
IJ = jimport('ij.IJ')
FileSaver = jimport('ij.io.FileSaver')
WaitForUserDialog = jimport('ij.gui.WaitForUserDialog')
ChannelSplitter = jimport('ij.plugin.ChannelSplitter')
ImageCalculator = jimport('ij.plugin.ImageCalculator')
NonBlockingGenericDialog = jimport('ij.gui.NonBlockingGenericDialog')
PointRoi = jimport('ij.gui.PointRoi')
GenericDialog = jimport('ij.gui.GenericDialog')
RoiManager = jimport('ij.plugin.frame.RoiManager')
WindowManager = jimport('ij.WindowManager')
Duplicator = jimport('ij.plugin.Duplicator')
ResultsTable = jimport('ij.measure.ResultsTable')
ImageStack = jimport('ij.ImageStack')
GaussianBlur = jimport('ij.plugin.filter.GaussianBlur')

2.14.0/1.54f


In [ ]:
#Search for all file.tif in the indicated folder -- copy-paste your path
path = r"D:\Quan\microscope\240412\2# emGFP_PMLwt_500ng"
files_paths = glob(os.path.join(path, "**", "*.tif"), recursive=True)
dre = re.compile(r'(\d+)')
files_paths.sort(key=lambda l: [int(s) if s.isdigit() else s.lower() for s in re.split(dre, l)])
e=0

#definition of output
df_results = pd.DataFrame(columns=['expmt', 'Nb of cells', 'Nb of transfected cells', 'Nb of diffuse cells', 'Nb of cells with condensates', 'Transfected/all', 'diffuse/transfected', 'condensates/transfected'])
df_data = pd.DataFrame(columns=['expmt', 'Nb of cells', 'Nb of transfected cells', 'Nb of diffuse cells', 'Nb of cells with condensates', 'Transfected/all', 'diffuse/transfected', 'condensates/transfected'])
results =[]

for filepath in files_paths:
    rm=RoiManager.getRoiManager()
    rm.reset()
    e += 1

    #Open the image
    imp = IJ.openImage(filepath)
    imp.show()

    #If z-stack then max intensity projection
    if (imp.getDimensions()[3]>1):
        IJ.run("Z Project...", "projection=[Max Intensity]")
        imp.close()
        imp.show()
        imp= IJ.getImage()

    # Reset contrast
    imp.setC(1)
    IJ.resetMinAndMax(imp)
    IJ.run(imp, "Enhance Contrast", "saturated=0.35")
    IJ.run(imp, "Next Slice [>]", "")
    IJ.resetMinAndMax(imp)
    imp.setC(1)

    # Reference experiment
    gui=NonBlockingGenericDialog("Experiment?")
    gui.addMessage("Ref+Lamella")
    # Replace XXX by ref experiment and x by lamella number
    gui.addStringField("n°",f'{e}')
    gui.enableYesNoCancel('ok', 'skip')
    gui.showDialog()

    if gui.wasCanceled():
        imp.close()
        break
    if not gui.wasOKed():
        imp.close()
        continue
    expmt=ij.py.from_java(gui.getNextString())

    # Adjust brighness and contrast manually
    IJ.run("Brightness/Contrast...")
    gui=NonBlockingGenericDialog("Adjust Brightness/Contrast")
    gui.enableYesNoCancel('ok', 'skip')
    gui.showDialog()

    # Select all cells
    rm.reset()
    IJ.run(imp, "Select None", "");
    IJ.setTool("multipoint")
    imp.setDisplayMode(IJ.COLOR)
    imp.setC(2)

    WaitForUserDialog("Select all cells and press 't'").show()
    all_cells_roi=rm.getRoisAsArray()

    if not all_cells_roi:
        WaitForUserDialog("Empty ROI Manager. Press 't' key after selecting cells. One more chance:").show()
        all_cells_roi=rm.getRoisAsArray()
        all_cells=all_cells_roi[0].getContainedPoints()

    all_cells=all_cells_roi[0].getContainedPoints()
    nb_cells = len(all_cells)

    # Select transfected diffuse cells
    rm.reset()
    IJ.run(imp, "Select None", "");
    imp.setDisplayMode(IJ.COLOR)
    imp.setC(1)

    WaitForUserDialog("Select transfected cells with diffuse fluorescence and press 't'").show()
    diffuse_cells_roi=rm.getRoisAsArray()

    if not diffuse_cells_roi:
        #WaitForUserDialog("Empty ROI Manager. Press 't' key after selecting cells. One more chance:").show()
        #diffuse_cells_roi=rm.getRoisAsArray()
        #diffuse_cells=diffuse_cells_roi[0].getContainedPoints()
        nb_diffuse_cells=0
    else:        
        diffuse_cells=diffuse_cells_roi[0].getContainedPoints()
        nb_diffuse_cells = len(diffuse_cells)

    # Select transfected cells with condensates
    rm.reset()
    IJ.run(imp, "Select None", "");
    imp.setDisplayMode(IJ.COLOR)
    imp.setC(1)

    WaitForUserDialog("Select transfected cells with condensates and press 't'").show()
    body_cells_roi=rm.getRoisAsArray()

    if not body_cells_roi:
        #WaitForUserDialog("Empty ROI Manager. Press 't' key after selecting cells. One more chance:").show()
        #body_cells_roi=rm.getRoisAsArray()
        #body_cells=body_cells_roi[0].getContainedPoints()
        nb_body_cells = 0
    else:
        body_cells=body_cells_roi[0].getContainedPoints()
        nb_body_cells = len(body_cells)

    # Storage of results from the current frame
    transfected_cells = nb_diffuse_cells + nb_body_cells
    data = [expmt, nb_cells, transfected_cells, nb_diffuse_cells, nb_body_cells, transfected_cells/nb_cells, nb_diffuse_cells/transfected_cells, nb_body_cells/transfected_cells]

    #Add results from the current frame to the big data table
    #results += [data]
    #Use dataframe
    df_data = pd.DataFrame([data], columns=['expmt', 'Nb of cells', 'Nb of transfected cells', 'Nb of diffuse cells', 'Nb of cells with condensates', 'Transfected/all', 'diffuse/transfected', 'condensates/transfected'])
    df_results = pd.concat([df_results, df_data], ignore_index=True)
display(df_results)

In [ ]:
#save as an excel file
df_results.to_excel(r'D:\Quan\data\2# emGFP_PMLwt_500ng.xlsx', index=False)